Importations

In [13]:
import nltk
import numpy as np
import random
import string # to process standard python strings
import warnings
warnings.filterwarnings('ignore')


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Lecture de donnees du fichier

In [15]:
f=open('/content/drive/My Drive/chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()# converts to lowercase
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


sent tokens words tokens

In [16]:
sent_tokens[:2]
['un chatbot (également appelé talkbot, chatterbot, bot, im bot, agent interactif ou entité conversationnelle artificielle) est un programme informatique ou une intelligence artificielle qui mène une conversation via des méthodes auditives ou textuelles. ',
 '«ces programmes sont souvent conçus pour simuler de manière convaincante comment un humain se comporterait comme un partenaire conversationnel, réussissant ainsi le test de turing.»']
word_tokens[:2]
['a', 'chatbot', '(', 'also', 'known']

['a', 'chatbot', '(', 'also', 'known']

Pre-processing the raw text

In [17]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

Keyword matching

In [29]:
GREETING_INPUTS = ("bonjour", "salut", "quoi de neuf", "salam", "bienvenu")
GREETING_RESPONSES = ["Salam aleyk! En quoi puis-je vous aider"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

Generating Response

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer


import cosine

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

Response

In [32]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Desole! je ne vous compremd pas"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

Feeding lines

In [33]:
flag=True
print("ROBO: je me nomme Robo. je vous repondrai sur vos questions concernant le Racisme. si vous voulez quitter, taper bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='merci' or user_response=='je vous remercie' ):
            flag=False
            print("ROBO: Je vous en prie ^^")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO:  au revoir!..")

ROBO: je me nomme Robo. je vous repondrai sur vos questions concernant le Racisme. si vous voulez quitter, taper bye!
,erci
ROBO: Desole! je ne vous compremd pas
merci
ROBO: Je vous en prie ^^
